## Using helix_funcs

An example of how to use the helix_funcs module to process the HelixScope datafiles, and generate a master output table.

Process_files should generate a similarly named csv file in the process folder.
combine_processed 

In [1]:
import helix_funcs
import geopandas as gpd
from tqdm import tqdm

In [3]:
s = gpd.read_file('./data/gadm28_adm1/gadm28_adm1.shp')
s = s.to_crs(epsg='4326')

In [4]:
fs = helix_funcs.identify_netcdf_and_csv_files()

In [5]:
for f in tqdm(fs['nc'][0:4]):
    helix_funcs.process_file(file=f, shps=s)

100%|██████████| 4/4 [00:00<00:00, 579.60it/s]


In [6]:
helix_funcs.combine_processed_results()

Generated ./master_admin1.csv: 12,600 rows of data from 4 sources.
